In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/simple-housing-price-prediction/sample_submission.csv
/kaggle/input/simple-housing-price-prediction/train.csv
/kaggle/input/simple-housing-price-prediction/test.csv


In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

In [3]:
train_data_path = "../input/simple-housing-price-prediction/train.csv"
test_data_path = "../input/simple-housing-price-prediction/test.csv"

df = pd.read_csv(train_data_path)
df_test = pd.read_csv(test_data_path)


In [4]:
X = df.copy()
y = X.pop('price')
X_test = df_test.copy()
X.head()

,house_id,date,location,type,block,street,storey_range,area_sqm,flat_model,commence_date
0,199577,2006-09,Raleigh,5 ROOM,107D,Agawan Court,07 TO 09,110.0,D,2003
1,217021,2007-06,Fresno,3 ROOM,678,Cleo St,07 TO 09,64.0,N,1988
2,308062,2010-09,Tucson,4 ROOM,5,E Pleasant View Way,10 TO 12,92.0,K,1976
3,212465,2007-04,Austin,4 ROOM,326,Park Hollow Ln,10 TO 12,92.0,K,1977
4,60654,2001-10,Honolulu,4 ROOM,794,Ala Puawa Place,04 TO 06,102.0,G,1998


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y,train_size=0.8, test_size=0.2,
                                                                random_state=0)



In [6]:
categorical_cols = [cname for cname in X_train.columns if
                    X_train[cname].nunique() < 10 and 
                    X_train[cname].dtype == "object"]

numerical_cols = [cname for cname in X_train.columns if 
                X_train[cname].dtype in ['int64', 'float64']]

my_cols = categorical_cols + numerical_cols
X_train = X_train[my_cols].copy()
X_valid = X_valid[my_cols].copy()
X_test = X_test[my_cols].copy()

In [7]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
])

In [8]:
# import optuna
# X = X_train
# def score_dataset(X, y, model=XGBRegressor()):
#     # Label encoding for categoricals
#     #
#     # Label encoding is good for XGBoost and RandomForest, but one-hot
#     # would be better for models like Lasso or Ridge. The `cat.codes`
#     # attribute holds the category levels.
#     for colname in X.select_dtypes(["category"]):
#         X[colname] = X[colname].cat.codes
#     # Metric for Housing competition is RMSLE (Root Mean Squared Log Error)
#     log_y = np.log(y)
#     score = cross_val_score(
#         model, X, log_y, cv=5, scoring="neg_mean_squared_error",
#     )
#     score = -1 * score.mean()
#     score = np.sqrt(score)
#     return score

# def objective(trial):
#     xgb_params = dict(
#         max_depth=trial.suggest_int("max_depth", 2, 10),
#         learning_rate=trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
#         n_estimators=trial.suggest_int("n_estimators", 1000, 8000),
#         min_child_weight=trial.suggest_int("min_child_weight", 1, 10),
#         colsample_bytree=trial.suggest_float("colsample_bytree", 0.2, 1.0),
#         subsample=trial.suggest_float("subsample", 0.2, 1.0),
#         reg_alpha=trial.suggest_float("reg_alpha", 1e-4, 1e2, log=True),
#         reg_lambda=trial.suggest_float("reg_lambda", 1e-4, 1e2, log=True),
#     )
#     xgb = XGBRegressor(**xgb_params)
#     return score_dataset(X_train, y_train, xgb)

# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=20)
# xgb_params = study.best_params

In [9]:
# Define model
model = XGBRegressor(n_estimators=4222,
                     learning_rate=0.001779955848977826,
                     max_depth=9,
                     min_child_weight=2,
                     random_state=0,
                     colsample_bytree=0.8068642787690588,
                     subsample=0.8493678320968241,
                     reg_alpha=0.031161768449367588,
                     reg_lambda=0.2657100756884015
                    )
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

preprocessor.fit_transform(X_train)
X_valid_transformed = preprocessor.transform(X_valid)
fit_params = {"model__eval_set": [(X_valid_transformed, y_valid)], 
              "model__early_stopping_rounds": 5,
              "model__verbose": False}

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train, **fit_params)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
print('MAE:', mean_absolute_error(y_valid, preds))

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


MAE: 29120.773242935753


In [10]:
# Preprocessing of test data, fit model
preds_test = my_pipeline.predict(X_test)

In [11]:
output = pd.DataFrame({'house_id': X_test.house_id,
                       'price': preds_test})
output.to_csv('submission.csv', index=False)